# NLP 07: Fuzzy Matching

Frequencies get me counts on exact matches. Now I'm interested in fuzzy matches, matches that are similar but not exact. What I want to get out of this are groups that are the same thing but are misspelled or abbreviated.

To get this information, I'll need to compare every word to every other word, which is computational expensive. I'll need to keep that in mind when I get to comparing addresses for all the rows. For example, while comparing each word to every other word in the 2,000+ row word list may be entirely feasible, doing the same for all 400,000+ addresses at the same time would take too long. Partitioning the data into chunks, perhaps smaller even that processing by country, will be necessary.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display
from time import gmtime, strftime
import sys
import os
import io

import string
import re
# import itertools
# import nltk
# nltk.download('stopwords')

from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from rapidfuzz import fuzz as rfuzz
import jaro

In [2]:
def frequency_ct(ngram_list):
    freq_dict = {}
    for ngram in ngram_list:
        if ngram not in freq_dict:
            freq_dict[ngram] = 0
        freq_dict[ngram] +=1
    return freq_dict

In [3]:
df = pd.read_csv('data/parsed_bahamas_addresses.csv')

df['address_wordlist'] = df['working_address'].fillna('').str.split()

freq_df = pd.DataFrame.from_dict(
    frequency_ct(df['address_wordlist'].sum()
                ), orient='index').reset_index().rename(
    columns={'index':'word', 0:'count'}).sort_values('count', ascending=False)

In [29]:
freq_df.head(3)

,word,count
9,bahamas,2324
8,nassau,2043
6,box,1484


## Fuzzy match metrics

There are two main ways to measure string similarity:

- Levenshtein Distance: uses the number of single characters edits needed to convert the first string in to the second
- Jaro-Winkler Distance: uses the number of matching characters and the number of transpositions

Edit operations include the following:


- Addition: Adding a character
- Deletion: Removing a character
- Substitution: Replacing a character
- Transposition: Swapping two adjacent characters

Levenshtein Distance uses the first three. An extension of this distance metric, Damerau-Levenshtein Distance, uses all four edit operations.

For more information on these metrics, Moosa Ali has a good write up on [Medium](https://medium.com/) in his [Best Libraries for Fuzzy Matching In Python](https://medium.com/codex/best-libraries-for-fuzzy-matching-in-python-cbb3e0ef87dd) article.

### `fuzzywuzzy`

The main library for performing fuzzy matching with python is the `fuzzywuzzy` package. It uses Levenshtein Distance to calculate how similar or dissimilar two strings are.

`fuzzywuzzy` comes with four metrics:

- Ratio: compares the entire string with the characters in order
- Partial ratio: compares the shorter string with a substring of the same length from the longer string
- Token sort ratio: compares the string while ignoring word/character order
- Token set ratio: compares the string while ignoring duplicate words/characters

For more information on these metrics, Catherine Gitau has a good explanation in her [Fuzzy String Matching](https://towardsdatascience.com/fuzzy-string-matching-in-python-68f240d910fe) article on [Towards Data Science](https://towardsdatascience.com/).

`fuzzywuzzy` also has several methods in the `.process` function:

- `extract`: compares a single string to a list of strings. When used with a series, returns three values
    - Second string that is being compared to the first string
    - Score
    - Index of the second string
- `dedup`: removes duplicate values from a list of strings from a specified score threshold

There are several limitations to these methods.

`extract` can only take a single string for the first value. I'd need to set this up in a for loop to compare every item to every other item. For loops are notoriously slow, so this doesn't appear to be a good solution given the amount of data I want to process.

`dedup` removes the duplicates without providing any information on what was removed. For my usecase, I'll need to know what is matching with what. For this initial example using the word list, I'll want to replace misspellings with the correct word to update the values in the `working_address` column. When I apply this to the full addresses, I'll need to group rows that should be the same address and give them a new node id. I can then use the original node id and the newly assigned node it to correctly associate addresses with their counterparts in the rest of the Offshore Leaks data.

### `rapidfuzz`

There is another package, `rapidfuzz` that is based on `fuzzywuzzy`, but supposedly runs faster. It also returns more detailed scores. `fuzzywuzzy` rounds to the nearest integer while `rapidfuzz` provides the decimal answer. FYI, occasionally they output different results.

In [10]:
str1 = freq_df.iloc[0,0]
str2 = freq_df.iloc[1,0]

print(f'Comparison strings: String 1: "{str1}", String 2: "{str2}"', '\n')
print('Metric', '\t\t\tfuzzywuzzy', '\trapidfuzz')
print('Ratio:', '\t\t\t', fuzz.ratio(str1, str2), '\t\t', rfuzz.ratio(str1, str2))
print('Partial ratio:', '\t\t', fuzz.partial_ratio(str1, str2), '\t\t', rfuzz.partial_ratio(str1, str2))
print('Token sort ratio:', '\t', fuzz.token_sort_ratio(str1, str2), '\t\t', rfuzz.token_sort_ratio(str1, str2))
print('Token set ratio:', '\t', fuzz.token_set_ratio(str1, str2), '\t\t', rfuzz.token_set_ratio(str1, str2))
print('Jaro-Winkler:', '\t', jaro.jaro_winkler_metric(str1, str2))

Comparison strings: String 1: "bahamas", String 2: "nassau" 

Metric 			fuzzywuzzy 	rapidfuzz
Ratio: 			 31 		 30.76923076923077
Partial ratio: 		 33 		 50.0
Token sort ratio: 	 31 		 30.76923076923077
Token set ratio: 	 31 		 30.769230769230774
Jaro-Winkler: 	 0.5396825396825397


## Storage format

Before deciding how to process all the data, I need to know what information to keep. Based on what I wanted to see above I need:

- The original string
- The match string
- All five metrics

Additionally, since I'm working with dataframes, I'll also want the index of both the original and match values so that I can easily find them in the dataframe.

Ultimately, when I'm working with the full address string, it should look something like this:

<table>
    <tr>
        <td>address_index</td>
        <td>address</td>
        <td>match_index</td>
        <td>match</td>
        <td>ratio_score</td>
        <td>partial_ratio_score</td>
        <td>token_sort_score</td>
        <td>token_set_score</td>
        <td>jaro_winkler_score</td>
    </tr>
    <tr>
        <td>9</td>
        <td>'ground floor goodmans bay corporate ce po box n 3933 nassau bahamas'</td>
        <td>1975</td>
        <td>'ground floor goodmans bay corporate ce po box n 3933 nassau bahamas'</td>
        <td>100</td>
        <td>100</td>
        <td>100</td>
        <td>100</td>
        <td>1.0</td>
    </tr>
    <tr>
        <td>9</td>
        <td>'ground floor goodmans bay corporate ce po box n 3933 nassau bahamas'</td>
        <td>2068</td>
        <td>'goodmans bay corporate centre po box cb10976 nassau bahamas'</td>
        <td>78</td>
        <td>87</td>
        <td>75</td>
        <td>85</td>
        <td>0.78</td>
    </tr>
    <tr>
        <td>9</td>
        <td>'ground floor goodmans bay corporate ce po box n 3933 nassau bahamas'</td>
        <td>548</td>
        <td>'second  floor goodmans bay corporate centre suite 261 po box cb12762 nassau bahamas'</td>
        <td>77</td>
        <td>69</td>
        <td>72</td>
        <td>85</td>
        <td>0.74</td>
    </tr>
    <tr>
        <td>9</td>
        <td>'ground floor goodmans bay corporate ce po box n 3933 nassau bahamas'</td>
        <td>268</td>
        <td>'goodmans bay corporate centre po box cb12407 nassau bahamas'</td>
        <td>76</td>
        <td>85</td>
        <td>75</td>
        <td>85</td>
        <td>0.78</td>
    </tr>
    <tr>
        <td>9</td>
        <td>'ground floor goodmans bay corporate ce po box n 3933 nassau bahamas'</td>
        <td>1511</td>
        <td>'goodmans bay corporate centre west bay street po box n3933 nassau bahamas'</td>
        <td>76</td>
        <td>70</td>
        <td>70</td>
        <td>79</td>
        <td>0.77</td>
    </tr>
    <tr>
        <td>9</td>
        <td>'ground floor goodmans bay corporate ce po box n 3933 nassau bahamas'</td>
        <td>1707</td>
        <td>'first floor goodmans bay corporate centre bay street nassau bahamas'</td>
        <td>76</td>
        <td>77</td>
        <td>73</td>
        <td>81</td>
        <td>0.73</td>
    </tr>
</table>

In [17]:
def calc_ffuzz_df(df, column):
    row_list = []
    
    for o_i, o_v in enumerate(df[column].sort_index()):
        for m_i, m_v in enumerate(df[column].sort_index()):
            if o_i != m_i:
                dict1 = {
                    'original_index': o_i,
                    'original_value': o_v,
                    'match_index': m_i,
                    'match_value': m_v,
                    'ratio_score': fuzz.ratio(o_v, m_v),
                    'partial_ratio_score': fuzz.partial_ratio(o_v, m_v),
                    'token_sort_score': fuzz.token_sort_ratio(o_v, m_v),
                    'token_set_score': fuzz.token_set_ratio(o_v, m_v),
                    'jaro_winkler_score': jaro.jaro_winkler_metric(o_v, m_v)
                }
                if (dict1['ratio_score']>0) | (dict1['partial_ratio_score']>0) | (dict1['token_sort_score']>0) | (dict1['token_set_score']>0) | (dict1['jaro_winkler_score']>0):
                    row_list.append(dict1)
    score_df = pd.DataFrame(row_list)
        
    return score_df

def calc_rfuzz_df(df, column):
    row_list = []
    
    for o_i, o_v in enumerate(df[column].sort_index()):
        for m_i, m_v in enumerate(df[column].sort_index()):
            if o_i != m_i:
                dict1 = {
                    'original_index': o_i,
                    'original_value': o_v,
                    'match_index': m_i,
                    'match_value': m_v,
                    'ratio_score': rfuzz.ratio(o_v, m_v),
                    'partial_ratio_score': rfuzz.partial_ratio(o_v, m_v),
                    'token_sort_score': rfuzz.token_sort_ratio(o_v, m_v),
                    'token_set_score': rfuzz.token_set_ratio(o_v, m_v),
                    'jaro_winkler_score': jaro.jaro_winkler_metric(o_v, m_v)
                }
                if (dict1['ratio_score']>0) | (dict1['partial_ratio_score']>0) | (dict1['token_sort_score']>0) | (dict1['token_set_score']>0) | (dict1['jaro_winkler_score']>0):
                    row_list.append(dict1)
    score_df = pd.DataFrame(row_list)
        
    return score_df

\**Notes* on the `calc_fuzz_df` function:

1. I use a dictionary to iteratively collect the scores as they're run. Initially, I used `.iloc` to concatenate a row onto the dataframe. However, this was unacceptably slow (I was too impatient to even let it finish). 
1. I don't store any row where all metrics are 0. If any metric has a value greater than 0, the whole row is captured.

### Function speed

Running the `%%timeit` magic function reveals that the `rapidfuzz` library is indeed significantly faster. As such, I'll be using `rapidfuzz`.

In [15]:
%%timeit
calc_rfuzz_df(freq_df, 'word')

43.4 s ± 200 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
%%timeit
calc_ffuzz_df(freq_df, 'word')

2min 42s ± 934 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [19]:
fuzzy_words_df = calc_rfuzz_df(freq_df, 'word')
fuzzy_words_df['jaro_winkler_score'] = fuzzy_words_df['jaro_winkler_score']*100
fuzzy_words_df

,original_index,original_value,match_index,match_value,ratio_score,partial_ratio_score,token_sort_score,token_set_score,jaro_winkler_score
0,0,annex,1,frederick,14.285714,25.000000,14.285714,14.285714,43.703704
1,0,annex,2,and,50.000000,80.000000,50.000000,50.000000,68.888889
2,0,annex,3,shirley,16.666667,28.571429,16.666667,16.666667,44.761905
3,0,annex,4,street,18.181818,28.571429,18.181818,18.181818,45.555556
4,0,annex,6,box,25.000000,50.000000,25.000000,25.000000,0.000000
...,...,...,...,...,...,...,...,...,...
2272337,2038,2ntl,2029,tortola,36.363636,50.000000,36.363636,36.363636,59.523810
2272338,2038,2ntl,2031,switzerland,26.666667,33.333333,26.666667,26.666667,56.060606
2272339,2038,2ntl,2033,montagne,33.333333,50.000000,33.333333,33.333333,58.333333
2272340,2038,2ntl,2034,sterline,33.333333,50.000000,33.333333,33.333333,58.333333


In [25]:
index_col = 'original_index'
metric_cts = pd.DataFrame(fuzzy_words_df[index_col].unique(), columns=[index_col])

for metric in ['ratio_score', 'partial_ratio_score', 'token_sort_score', 'token_set_score', 'jaro_winkler_score']:
    met_df = fuzzy_words_df.loc[fuzzy_words_df[metric]>60, [index_col, metric]].groupby(index_col).count().reset_index()
    metric_cts = metric_cts.merge(met_df, on=index_col, how='outer')
    
metric_cts = fuzzy_words_df[[index_col, 'original_value']].drop_duplicates().merge(metric_cts, on=index_col, how='outer')
metric_cts.columns = ['original_index', 'original_value', 'ratio_match_ct', 'partial_ratio_match_ct', 'token_sort_match_ct', 'token_set_match_ct', 'jaro_winkler_match_ct']
metric_cts

,original_index,original_value,ratio_match_ct,partial_ratio_match_ct,token_sort_match_ct,token_set_match_ct,jaro_winkler_match_ct
0,0,annex,5.0,58,5.0,5.0,55
1,1,frederick,3.0,63,3.0,3.0,67
2,2,and,14.0,239,14.0,14.0,133
3,3,shirley,19.0,60,19.0,19.0,122
4,4,street,18.0,81,18.0,18.0,179
...,...,...,...,...,...,...,...
2034,2034,sterline,21.0,127,21.0,21.0,238
2035,2035,bav,7.0,97,7.0,7.0,91
2036,2036,hast,6.0,109,6.0,6.0,114
2037,2037,coj,1.0,100,1.0,1.0,84


In [26]:
metric_cts[metric_cts['original_value'].str.contains('^n\d+|cb\d+|no\d+|\d+$')]

,original_index,original_value,ratio_match_ct,partial_ratio_match_ct,token_sort_match_ct,token_set_match_ct,jaro_winkler_match_ct
7,7,n4805,16.0,113,16.0,16.0,123
11,11,e2,3.0,412,3.0,3.0,24
15,15,n8188,17.0,97,17.0,17.0,88
19,19,n7785,18.0,102,18.0,18.0,88
20,20,n3708,8.0,120,8.0,8.0,113
...,...,...,...,...,...,...,...
2019,2019,3242,24.0,121,24.0,24.0,137
2020,2020,25,13.0,252,13.0,13.0,55
2022,2022,875,6.0,114,6.0,6.0,58
2023,2023,cr567,2.0,39,2.0,2.0,20


In [27]:
metric_cts = metric_cts[~metric_cts['original_value'].str.contains('^n\d+|^\w\w\d+|^\d+$')]
metric_cts

,original_index,original_value,ratio_match_ct,partial_ratio_match_ct,token_sort_match_ct,token_set_match_ct,jaro_winkler_match_ct
0,0,annex,5.0,58,5.0,5.0,55
1,1,frederick,3.0,63,3.0,3.0,67
2,2,and,14.0,239,14.0,14.0,133
3,3,shirley,19.0,60,19.0,19.0,122
4,4,street,18.0,81,18.0,18.0,179
...,...,...,...,...,...,...,...
2034,2034,sterline,21.0,127,21.0,21.0,238
2035,2035,bav,7.0,97,7.0,7.0,91
2036,2036,hast,6.0,109,6.0,6.0,114
2037,2037,coj,1.0,100,1.0,1.0,84


### Street

In [30]:
fuzzy_words_df[(fuzzy_words_df['original_value']=='street') & (fuzzy_words_df['ratio_score']>50)].sort_values('ratio_score', ascending=False)

,original_index,original_value,match_index,match_value,ratio_score,partial_ratio_score,token_sort_score,token_set_score,jaro_winkler_score
5568,4,street,1198,streeet,92.307692,90.909091,92.307692,92.307692,97.142857
5171,4,street,322,stret,90.909091,88.888889,90.909091,90.909091,96.666667
5275,4,street,607,stree,90.909091,100.000000,90.909091,90.909091,96.666667
5993,4,street,1790,streeets,85.714286,90.909091,85.714286,85.714286,95.000000
5902,4,street,1629,strets,83.333333,90.909091,83.333333,83.333333,93.333333
5556,4,street,1179,streer,83.333333,90.909091,83.333333,83.333333,93.333333
5870,4,street,1579,steret,83.333333,83.333333,83.333333,83.333333,95.555556
5428,4,street,947,strees,83.333333,90.909091,83.333333,83.333333,93.333333
6084,4,street,1968,stre,80.000000,100.000000,80.000000,80.000000,93.333333
5971,4,street,1743,treetops,71.428571,90.909091,71.428571,71.428571,81.944444


In [31]:
fuzzy_words_df[(fuzzy_words_df['original_value']=='street') & (fuzzy_words_df['partial_ratio_score']>50)].sort_values('partial_ratio_score', ascending=False)

,original_index,original_value,match_index,match_value,ratio_score,partial_ratio_score,token_sort_score,token_set_score,jaro_winkler_score
5307,4,street,713,ee,50.000000,100.000000,50.000000,50.000000,55.555556
5975,4,street,1751,t,28.571429,100.000000,28.571429,28.571429,72.222222
5212,4,street,433,r,28.571429,100.000000,28.571429,28.571429,72.222222
5275,4,street,607,stree,90.909091,100.000000,90.909091,90.909091,96.666667
5101,4,street,123,s,28.571429,100.000000,28.571429,28.571429,75.000000
...,...,...,...,...,...,...,...,...,...
5760,4,street,1442,systems,46.153846,54.545455,46.153846,46.153846,64.285714
5759,4,street,1441,netware,46.153846,54.545455,46.153846,46.153846,53.174603
5748,4,street,1429,microsystems,33.333333,54.545455,33.333333,33.333333,50.000000
5709,4,street,1380,ventures,42.857143,54.545455,42.857143,42.857143,63.888889


In [32]:
fuzzy_words_df[(fuzzy_words_df['original_value']=='street') & (fuzzy_words_df['token_sort_score']>50)].sort_values('token_sort_score', ascending=False)

,original_index,original_value,match_index,match_value,ratio_score,partial_ratio_score,token_sort_score,token_set_score,jaro_winkler_score
5568,4,street,1198,streeet,92.307692,90.909091,92.307692,92.307692,97.142857
5171,4,street,322,stret,90.909091,88.888889,90.909091,90.909091,96.666667
5275,4,street,607,stree,90.909091,100.000000,90.909091,90.909091,96.666667
5993,4,street,1790,streeets,85.714286,90.909091,85.714286,85.714286,95.000000
5902,4,street,1629,strets,83.333333,90.909091,83.333333,83.333333,93.333333
5556,4,street,1179,streer,83.333333,90.909091,83.333333,83.333333,93.333333
5870,4,street,1579,steret,83.333333,83.333333,83.333333,83.333333,95.555556
5428,4,street,947,strees,83.333333,90.909091,83.333333,83.333333,93.333333
6084,4,street,1968,stre,80.000000,100.000000,80.000000,80.000000,93.333333
5971,4,street,1743,treetops,71.428571,90.909091,71.428571,71.428571,81.944444


In [33]:
fuzzy_words_df[(fuzzy_words_df['original_value']=='street') & (fuzzy_words_df['token_set_score']>50)].sort_values('token_set_score', ascending=False)

,original_index,original_value,match_index,match_value,ratio_score,partial_ratio_score,token_sort_score,token_set_score,jaro_winkler_score
5568,4,street,1198,streeet,92.307692,90.909091,92.307692,92.307692,97.142857
5171,4,street,322,stret,90.909091,88.888889,90.909091,90.909091,96.666667
5275,4,street,607,stree,90.909091,100.000000,90.909091,90.909091,96.666667
5993,4,street,1790,streeets,85.714286,90.909091,85.714286,85.714286,95.000000
5902,4,street,1629,strets,83.333333,90.909091,83.333333,83.333333,93.333333
5556,4,street,1179,streer,83.333333,90.909091,83.333333,83.333333,93.333333
5870,4,street,1579,steret,83.333333,83.333333,83.333333,83.333333,95.555556
5428,4,street,947,strees,83.333333,90.909091,83.333333,83.333333,93.333333
6084,4,street,1968,stre,80.000000,100.000000,80.000000,80.000000,93.333333
5971,4,street,1743,treetops,71.428571,90.909091,71.428571,71.428571,81.944444


In [34]:
fuzzy_words_df[(fuzzy_words_df['original_value']=='street') & (fuzzy_words_df['jaro_winkler_score']>50)].sort_values('jaro_winkler_score', ascending=False)

,original_index,original_value,match_index,match_value,ratio_score,partial_ratio_score,token_sort_score,token_set_score,jaro_winkler_score
5568,4,street,1198,streeet,92.307692,90.909091,92.307692,92.307692,97.142857
5171,4,street,322,stret,90.909091,88.888889,90.909091,90.909091,96.666667
5275,4,street,607,stree,90.909091,100.000000,90.909091,90.909091,96.666667
5870,4,street,1579,steret,83.333333,83.333333,83.333333,83.333333,95.555556
5993,4,street,1790,streeets,85.714286,90.909091,85.714286,85.714286,95.000000
...,...,...,...,...,...,...,...,...,...
5788,4,street,1478,consultores,47.058824,54.545455,47.058824,47.058824,50.505051
5647,4,street,1304,consultants,35.294118,36.363636,35.294118,35.294118,50.505051
5553,4,street,1171,redomiciled,35.294118,50.000000,35.294118,35.294118,50.505051
5691,4,street,1356,association,23.529412,33.333333,23.529412,23.529412,50.505051


In [35]:
word_list = fuzzy_words_df[(fuzzy_words_df['original_value']=='street') & (fuzzy_words_df['jaro_winkler_score']>75)].sort_values('jaro_winkler_score', ascending=False)['match_value']
for word in word_list:
    print(word)
    display(df[df['address_wordlist'].apply(lambda x: word in x)])
    print('\n')

streeet


,node_id,address,name,countries,country_codes,sourceID,valid_until,note,working_address,address_wordlist
1031,14079989,Suite E-2; Union Court Building; Elizabeth Ave...,NaN,Bahamas,BHS,Panama Papers,The Panama Papers data is current through 2015,NaN,suite e 2 union court building elizabeth avenu...,"[suite, e, 2, union, court, building, elizabet..."




stret


,node_id,address,name,countries,country_codes,sourceID,valid_until,note,working_address,address_wordlist
156,24000157,"SG HAMBROS BUILDING WEST BAY STRET, P.O. BOX C...",NaN,Bahamas,BHS,Bahamas Leaks,The Bahamas Leaks data is current through earl...,NaN,sg hambros building west bay stret po box cb12...,"[sg, hambros, building, west, bay, stret, po, ..."
694,14035596,Charlotte House; Charlotte Stret; POB N-65; na...,NaN,Bahamas,BHS,Panama Papers,The Panama Papers data is current through 2015,NaN,charlotte house charlotte stret pob n 65 nassa...,"[charlotte, house, charlotte, stret, pob, n, 6..."
1383,286502,"SUITE 306, 3/F CENTRE OF COMMERCE 1 BAY STRET ...",NaN,Bahamas,BHS,Offshore Leaks,The Offshore Leaks data is current through 2010,NaN,suite 306 3f centre of commerce 1 bay stret na...,"[suite, 306, 3f, centre, of, commerce, 1, bay,..."




stree


...

,node_id,address,name,countries,country_codes,sourceID,valid_until,note,working_address,address_wordlist
1533,81021574,90 Summerset House; Thomson Blvd.; NASSAU; Bah...,90 Summerset House,Bahamas,BHS,Paradise Papers - Appleby,Appleby data is current through 2014,NaN,90 summerset house thomson boulevard nassau ba...,"[90, summerset, house, thomson, boulevard, nas..."




sommerset


,node_id,address,name,countries,country_codes,sourceID,valid_until,note,working_address,address_wordlist
1801,81013692,Sommerset House; Thomson Blvd.; NASSAU; Bahamas,Sommerset House,Bahamas,BHS,Paradise Papers - Appleby,Appleby data is current through 2014,NaN,sommerset house thomson boulevard nassau bahamas,"[sommerset, house, thomson, boulevard, nassau,..."




s


,node_id,address,name,countries,country_codes,sourceID,valid_until,note,working_address,address_wordlist
46,24000047,"SUITE D, S.G. HAMBROS BLDG., P.O. BOX N-3741, ...",NaN,Bahamas,BHS,Bahamas Leaks,The Bahamas Leaks data is current through earl...,NaN,suite d s g hambros bldg po box n3741 nassau b...,"[suite, d, s, g, hambros, bldg, po, box, n3741..."
213,24000214,"F.E.S. BUILDING - MILTON STREET, P.O. BOX F-44...",NaN,Bahamas,BHS,Bahamas Leaks,The Bahamas Leaks data is current through earl...,NaN,f e s building milton street po box f44181 fre...,"[f, e, s, building, milton, street, po, box, f..."
307,24000308,"SUITE 1, K. S. DARLING BUILDING, P.O. BOX N-49...",NaN,Bahamas,BHS,Bahamas Leaks,The Bahamas Leaks data is current through earl...,NaN,suite 1 k s darling building po box n4922 dowd...,"[suite, 1, k, s, darling, building, po, box, n..."
493,24000494,"K S DARLING BLDG. DOWDESWELL STREET, P.O. BOX ...",NaN,Bahamas,BHS,Bahamas Leaks,The Bahamas Leaks data is current through earl...,NaN,k s darling bldg dowdeswell street po box n948...,"[k, s, darling, bldg, dowdeswell, street, po, ..."
759,14043537,GOODMAN S BAY CORPORATE CENTER - WEST BAY STRE...,NaN,Bahamas,BHS,Panama Papers,The Panama Papers data is current through 2015,NaN,goodman s bay corporate center west bay street...,"[goodman, s, bay, corporate, center, west, bay..."
760,14043538,GOODMAN S BAY CORPORATE CENTER WEST BAY STREE...,NaN,Bahamas,BHS,Panama Papers,The Panama Papers data is current through 2015,NaN,goodman s bay corporate center west bay street...,"[goodman, s, bay, corporate, center, west, bay..."
928,14077070,SAFFREY SQUARE; SUITE 205; BANK LANE P.O. BOX ...,NaN,Bahamas,BHS,Panama Papers,The Panama Papers data is current through 2015,NaN,saffrey square suite 205 bank lane po box n818...,"[saffrey, square, suite, 205, bank, lane, po, ..."
1129,14081728,TRUSBAN INTERNATIONAL S.A. SUITE E-2 UNION COU...,NaN,Bahamas,BHS,Panama Papers,The Panama Papers data is current through 2015,NaN,trusban international s a suite e 2 union cour...,"[trusban, international, s, a, suite, e, 2, un..."
1200,14090904,ADMINOTIS S.A. SUITE E-2 UNION COURT BUILDING ...,NaN,Bahamas,BHS,Panama Papers,The Panama Papers data is current through 2015,NaN,adminotis s a suite e 2 union court building e...,"[adminotis, s, a, suite, e, 2, union, court, b..."
1208,14091417,ANATOLE TRADING S.A. SUITE E-2 UNION COURT BUI...,NaN,Bahamas,BHS,Panama Papers,The Panama Papers data is current through 2015,NaN,anatole trading s a suite e 2 union court buil...,"[anatole, trading, s, a, suite, e, 2, union, c..."


### Bahamas

In [36]:
fuzzy_words_df[(fuzzy_words_df['original_value']=='bahamas') & (fuzzy_words_df['ratio_score']>60)].sort_values('ratio_score', ascending=False)

,original_index,original_value,match_index,match_value,ratio_score,partial_ratio_score,token_sort_score,token_set_score,jaro_winkler_score
10544,9,bahamas,1101,bahamaas,93.333333,92.307692,93.333333,93.333333,97.500000
11087,9,bahamas,1911,abahamas,93.333333,100.000000,93.333333,93.333333,81.547619
10909,9,bahamas,1602,bahamasa,93.333333,100.000000,93.333333,93.333333,97.500000
10603,9,bahamas,1200,bahamas6,93.333333,100.000000,93.333333,93.333333,97.500000
10436,9,bahamas,930,bahamas1,93.333333,100.000000,93.333333,93.333333,97.500000
10421,9,bahamas,907,bahamasc,93.333333,100.000000,93.333333,93.333333,97.500000
10424,9,bahamas,914,ahamas,92.307692,100.000000,92.307692,92.307692,95.238095
10107,9,bahamas,188,bahama,92.307692,100.000000,92.307692,92.307692,97.142857
10420,9,bahamas,904,bahmas,92.307692,83.333333,92.307692,92.307692,92.777778
10300,9,bahamas,659,bhamas,92.307692,90.909091,92.307692,92.307692,85.714286


In [37]:
fuzzy_words_df[(fuzzy_words_df['original_value']=='bahamas') & (fuzzy_words_df['partial_ratio_score']>70)].sort_values('partial_ratio_score', ascending=False)

,original_index,original_value,match_index,match_value,ratio_score,partial_ratio_score,token_sort_score,token_set_score,jaro_winkler_score
10543,9,bahamas,1100,ba,44.444444,100.000000,44.444444,44.444444,80.952381
10367,9,bahamas,806,bah,60.000000,100.000000,60.000000,60.000000,86.666667
10603,9,bahamas,1200,bahamas6,93.333333,100.000000,93.333333,93.333333,97.500000
11087,9,bahamas,1911,abahamas,93.333333,100.000000,93.333333,93.333333,81.547619
10551,9,bahamas,1125,bahamaspo,87.500000,100.000000,87.500000,87.500000,95.555556
10082,9,bahamas,123,s,25.000000,100.000000,25.000000,25.000000,0.000000
10436,9,bahamas,930,bahamas1,93.333333,100.000000,93.333333,93.333333,97.500000
10432,9,bahamas,926,as,44.444444,100.000000,44.444444,44.444444,54.761905
10424,9,bahamas,914,ahamas,92.307692,100.000000,92.307692,92.307692,95.238095
10780,9,bahamas,1425,m,25.000000,100.000000,25.000000,25.000000,0.000000


In [38]:
fuzzy_words_df[(fuzzy_words_df['original_value']=='bahamas') & (fuzzy_words_df['token_sort_score']>65)].sort_values('token_sort_score', ascending=False)

,original_index,original_value,match_index,match_value,ratio_score,partial_ratio_score,token_sort_score,token_set_score,jaro_winkler_score
10544,9,bahamas,1101,bahamaas,93.333333,92.307692,93.333333,93.333333,97.500000
11087,9,bahamas,1911,abahamas,93.333333,100.000000,93.333333,93.333333,81.547619
10909,9,bahamas,1602,bahamasa,93.333333,100.000000,93.333333,93.333333,97.500000
10421,9,bahamas,907,bahamasc,93.333333,100.000000,93.333333,93.333333,97.500000
10603,9,bahamas,1200,bahamas6,93.333333,100.000000,93.333333,93.333333,97.500000
10436,9,bahamas,930,bahamas1,93.333333,100.000000,93.333333,93.333333,97.500000
10233,9,bahamas,516,bahams,92.307692,90.909091,92.307692,92.307692,97.142857
10107,9,bahamas,188,bahama,92.307692,100.000000,92.307692,92.307692,97.142857
10424,9,bahamas,914,ahamas,92.307692,100.000000,92.307692,92.307692,95.238095
10420,9,bahamas,904,bahmas,92.307692,83.333333,92.307692,92.307692,92.777778


In [39]:
fuzzy_words_df[(fuzzy_words_df['original_value']=='bahamas') & (fuzzy_words_df['token_set_score']>65)].sort_values('token_set_score', ascending=False)

,original_index,original_value,match_index,match_value,ratio_score,partial_ratio_score,token_sort_score,token_set_score,jaro_winkler_score
10544,9,bahamas,1101,bahamaas,93.333333,92.307692,93.333333,93.333333,97.500000
11087,9,bahamas,1911,abahamas,93.333333,100.000000,93.333333,93.333333,81.547619
10909,9,bahamas,1602,bahamasa,93.333333,100.000000,93.333333,93.333333,97.500000
10421,9,bahamas,907,bahamasc,93.333333,100.000000,93.333333,93.333333,97.500000
10603,9,bahamas,1200,bahamas6,93.333333,100.000000,93.333333,93.333333,97.500000
10436,9,bahamas,930,bahamas1,93.333333,100.000000,93.333333,93.333333,97.500000
10233,9,bahamas,516,bahams,92.307692,90.909091,92.307692,92.307692,97.142857
10107,9,bahamas,188,bahama,92.307692,100.000000,92.307692,92.307692,97.142857
10424,9,bahamas,914,ahamas,92.307692,100.000000,92.307692,92.307692,95.238095
10420,9,bahamas,904,bahmas,92.307692,83.333333,92.307692,92.307692,92.777778


In [40]:
fuzzy_words_df[(fuzzy_words_df['original_value']=='bahamas') & (fuzzy_words_df['jaro_winkler_score']>75)].sort_values('jaro_winkler_score', ascending=False)

,original_index,original_value,match_index,match_value,ratio_score,partial_ratio_score,token_sort_score,token_set_score,jaro_winkler_score
10421,9,bahamas,907,bahamasc,93.333333,100.000000,93.333333,93.333333,97.500000
10909,9,bahamas,1602,bahamasa,93.333333,100.000000,93.333333,93.333333,97.500000
10603,9,bahamas,1200,bahamas6,93.333333,100.000000,93.333333,93.333333,97.500000
10544,9,bahamas,1101,bahamaas,93.333333,92.307692,93.333333,93.333333,97.500000
10436,9,bahamas,930,bahamas1,93.333333,100.000000,93.333333,93.333333,97.500000
10107,9,bahamas,188,bahama,92.307692,100.000000,92.307692,92.307692,97.142857
10491,9,bahamas,1018,bahaams,85.714286,85.714286,85.714286,85.714286,97.142857
10233,9,bahamas,516,bahams,92.307692,90.909091,92.307692,92.307692,97.142857
10551,9,bahamas,1125,bahamaspo,87.500000,100.000000,87.500000,87.500000,95.555556
10424,9,bahamas,914,ahamas,92.307692,100.000000,92.307692,92.307692,95.238095


In [41]:
word_list = fuzzy_words_df[(fuzzy_words_df['original_value']=='bahamas') & (fuzzy_words_df['jaro_winkler_score']>75)].sort_values('jaro_winkler_score', ascending=False)['match_value']
for word in word_list:
    print(word)
    display(df[df['address_wordlist'].apply(lambda x: word in x)])
    print('\n')

bahamasc


,node_id,address,name,countries,country_codes,sourceID,valid_until,note,working_address,address_wordlist
570,14018521,51 Frederick Street; P.O. Box N-1136; Nassau; ...,NaN,Bahamas,BHS,Panama Papers,The Panama Papers data is current through 2015,NaN,51 frederick street po box n1136 nassau bahamasc,"[51, frederick, street, po, box, n1136, nassau..."




bahamasa


,node_id,address,name,countries,country_codes,sourceID,valid_until,note,working_address,address_wordlist
1442,240054,Winterbotham Place Marlborough & Queen Streets...,NaN,Bahamas,BHS,Offshore Leaks,The Offshore Leaks data is current through 2010,NaN,winterbotham place marlborough and queen stree...,"[winterbotham, place, marlborough, and, queen,..."




bahamas6


...

,node_id,address,name,countries,country_codes,sourceID,valid_until,note,working_address,address_wordlist
549,14012324,2nd Floor; International Bazaar; Bay Street; P...,NaN,Bahamas,BHS,Panama Papers,The Panama Papers data is current through 2015,NaN,second floor international bazaar bay street p...,"[second, floor, international, bazaar, bay, st..."




bosham


,node_id,address,name,countries,country_codes,sourceID,valid_until,note,working_address,address_wordlist
199,24000200,"#6 BOSHAM CLOSE, CAMPERDOWN HEIGHTS P.O. BOX S...",NaN,Bahamas,BHS,Bahamas Leaks,The Bahamas Leaks data is current through earl...,NaN,6 bosham close camperdown heights po box sp 63...,"[6, bosham, close, camperdown, heights, po, bo..."
1887,33000077,"#6 BOSHAM CLOSE, CAMPERDOWN HEIGHTS PO BOX SP ...",NaN,Bahamas,BHS,Paradise Papers - Bahamas corporate registry,Bahamas corporate registry data is current thr...,NaN,6 bosham close camperdown heights po box sp 63...,"[6, bosham, close, camperdown, heights, po, bo..."
2180,240003759,"NO. 6 BOSHAM CLOSE, CAMPERDOWN HEIGHTS NEW PRO...",NaN,Bahamas,BHS,"Pandora Papers - Alemán, Cordero, Galindo & Le...",Provider data is current through 2018,NaN,no 6 bosham close camperdown heights new provi...,"[no, 6, bosham, close, camperdown, heights, ne..."




hamas


,node_id,address,name,countries,country_codes,sourceID,valid_until,note,working_address,address_wordlist
2243,240492153,"MONTAGUE STERLING CENTRE. EAST BAY STREET, NAS...",NaN,Bahamas,BHS,Pandora Papers - Trident Trust,Provider data is current through 2016,NaN,montague sterling centre east bay street nassa...,"[montague, sterling, centre, east, bay, street..."


### Nassau

In [42]:
fuzzy_words_df[(fuzzy_words_df['original_value']=='nassau') & (fuzzy_words_df['ratio_score']>60)].sort_values('ratio_score', ascending=False)

,original_index,original_value,match_index,match_value,ratio_score,partial_ratio_score,token_sort_score,token_set_score,jaro_winkler_score
9110,8,nassau,698,nasssau,92.307692,83.333333,92.307692,92.307692,97.142857
9248,8,nassau,925,nassaub,92.307692,100.000000,92.307692,92.307692,97.142857
9398,8,nassau,1160,nassaau,92.307692,90.909091,92.307692,92.307692,97.142857
9453,8,nassau,1224,nassaus,92.307692,100.000000,92.307692,92.307692,97.142857
9746,8,nassau,1593,naussau,92.307692,83.333333,92.307692,92.307692,96.190476
9217,8,nassau,872,nasau,90.909091,80.000000,90.909091,90.909091,96.111111
9424,8,nassau,1194,nassu,90.909091,88.888889,90.909091,90.909091,96.666667
9350,8,nassau,1083,massau,83.333333,90.909091,83.333333,83.333333,88.888889
9387,8,nassau,1146,nassua,83.333333,90.909091,83.333333,83.333333,96.666667
9415,8,nassau,1185,nassao,83.333333,90.909091,83.333333,83.333333,93.333333


In [43]:
fuzzy_words_df[(fuzzy_words_df['original_value']=='nassau') & (fuzzy_words_df['partial_ratio_score']>70)].sort_values('partial_ratio_score', ascending=False)

,original_index,original_value,match_index,match_value,ratio_score,partial_ratio_score,token_sort_score,token_set_score,jaro_winkler_score
8654,8,nassau,49,n,28.571429,100.000000,28.571429,28.571429,75.000000
8792,8,nassau,238,a,28.571429,100.000000,28.571429,28.571429,72.222222
8995,8,nassau,530,ss,50.000000,100.000000,50.000000,50.000000,77.777778
9853,8,nassau,1755,na,50.000000,100.000000,50.000000,50.000000,82.222222
9453,8,nassau,1224,nassaus,92.307692,100.000000,92.307692,92.307692,97.142857
9248,8,nassau,925,nassaub,92.307692,100.000000,92.307692,92.307692,97.142857
9249,8,nassau,926,as,50.000000,100.000000,50.000000,50.000000,77.777778
9290,8,nassau,993,343nassau,80.000000,100.000000,80.000000,80.000000,88.888889
8707,8,nassau,123,s,28.571429,100.000000,28.571429,28.571429,72.222222
9398,8,nassau,1160,nassaau,92.307692,90.909091,92.307692,92.307692,97.142857


In [44]:
fuzzy_words_df[(fuzzy_words_df['original_value']=='nassau') & (fuzzy_words_df['token_sort_score']>80)].sort_values('token_sort_score', ascending=False)

,original_index,original_value,match_index,match_value,ratio_score,partial_ratio_score,token_sort_score,token_set_score,jaro_winkler_score
9110,8,nassau,698,nasssau,92.307692,83.333333,92.307692,92.307692,97.142857
9248,8,nassau,925,nassaub,92.307692,100.000000,92.307692,92.307692,97.142857
9398,8,nassau,1160,nassaau,92.307692,90.909091,92.307692,92.307692,97.142857
9453,8,nassau,1224,nassaus,92.307692,100.000000,92.307692,92.307692,97.142857
9746,8,nassau,1593,naussau,92.307692,83.333333,92.307692,92.307692,96.190476
9217,8,nassau,872,nasau,90.909091,80.000000,90.909091,90.909091,96.111111
9424,8,nassau,1194,nassu,90.909091,88.888889,90.909091,90.909091,96.666667
9350,8,nassau,1083,massau,83.333333,90.909091,83.333333,83.333333,88.888889
9387,8,nassau,1146,nassua,83.333333,90.909091,83.333333,83.333333,96.666667
9415,8,nassau,1185,nassao,83.333333,90.909091,83.333333,83.333333,93.333333


In [45]:
fuzzy_words_df[(fuzzy_words_df['original_value']=='nassau') & (fuzzy_words_df['token_set_score']>75)].sort_values('token_set_score', ascending=False)

,original_index,original_value,match_index,match_value,ratio_score,partial_ratio_score,token_sort_score,token_set_score,jaro_winkler_score
9110,8,nassau,698,nasssau,92.307692,83.333333,92.307692,92.307692,97.142857
9248,8,nassau,925,nassaub,92.307692,100.000000,92.307692,92.307692,97.142857
9398,8,nassau,1160,nassaau,92.307692,90.909091,92.307692,92.307692,97.142857
9453,8,nassau,1224,nassaus,92.307692,100.000000,92.307692,92.307692,97.142857
9746,8,nassau,1593,naussau,92.307692,83.333333,92.307692,92.307692,96.190476
9217,8,nassau,872,nasau,90.909091,80.000000,90.909091,90.909091,96.111111
9424,8,nassau,1194,nassu,90.909091,88.888889,90.909091,90.909091,96.666667
9350,8,nassau,1083,massau,83.333333,90.909091,83.333333,83.333333,88.888889
9387,8,nassau,1146,nassua,83.333333,90.909091,83.333333,83.333333,96.666667
9415,8,nassau,1185,nassao,83.333333,90.909091,83.333333,83.333333,93.333333


In [46]:
fuzzy_words_df[(fuzzy_words_df['original_value']=='nassau') & (fuzzy_words_df['jaro_winkler_score']>85)].sort_values('jaro_winkler_score', ascending=False)

,original_index,original_value,match_index,match_value,ratio_score,partial_ratio_score,token_sort_score,token_set_score,jaro_winkler_score
9110,8,nassau,698,nasssau,92.307692,83.333333,92.307692,92.307692,97.142857
9248,8,nassau,925,nassaub,92.307692,100.000000,92.307692,92.307692,97.142857
9398,8,nassau,1160,nassaau,92.307692,90.909091,92.307692,92.307692,97.142857
9453,8,nassau,1224,nassaus,92.307692,100.000000,92.307692,92.307692,97.142857
9387,8,nassau,1146,nassua,83.333333,90.909091,83.333333,83.333333,96.666667
9424,8,nassau,1194,nassu,90.909091,88.888889,90.909091,90.909091,96.666667
9746,8,nassau,1593,naussau,92.307692,83.333333,92.307692,92.307692,96.190476
9217,8,nassau,872,nasau,90.909091,80.000000,90.909091,90.909091,96.111111
9415,8,nassau,1185,nassao,83.333333,90.909091,83.333333,83.333333,93.333333
9431,8,nassau,1201,nassan,83.333333,90.909091,83.333333,83.333333,93.333333


In [47]:
word_list = fuzzy_words_df[(fuzzy_words_df['original_value']=='nassau') & (fuzzy_words_df['jaro_winkler_score']>85)].sort_values('jaro_winkler_score', ascending=False)['match_value']
for word in word_list:
    print(word)
    display(df[df['address_wordlist'].apply(lambda x: word in x)])
    print('\n')

nasssau


,node_id,address,name,countries,country_codes,sourceID,valid_until,note,working_address,address_wordlist
420,24000421,"3RD FLOOR, GEORGE HOUSE, GEORGE STREET, P.O. B...",NaN,Bahamas,BHS,Bahamas Leaks,The Bahamas Leaks data is current through earl...,NaN,third floor george house george street po box ...,"[third, floor, george, house, george, street, ..."




nassaub


,node_id,address,name,countries,country_codes,sourceID,valid_until,note,working_address,address_wordlist
583,14026897,ANSBACHER (BAHAMAS) LIMITED P.O. BOX N 7768 AN...,NaN,Bahamas,BHS,Panama Papers,The Panama Papers data is current through 2015,NaN,ansbacher bahamas limited po box n 7768 ansbac...,"[ansbacher, bahamas, limited, po, box, n, 7768..."




nassaau


...

,node_id,address,name,countries,country_codes,sourceID,valid_until,note,working_address,address_wordlist
1043,14080003,Suite E-2; Union Court Building; Elizabeth Ave...,NaN,Bahamas,BHS,Panama Papers,The Panama Papers data is current through 2015,NaN,suite e 2 union court building elizabeth avenu...,"[suite, e, 2, union, court, building, elizabet..."
1050,14080011,Suite E-2; Union Court Building; Elizabeth Ave...,NaN,Bahamas,BHS,Panama Papers,The Panama Papers data is current through 2015,NaN,suite e 2 union court building elizabeth avenu...,"[suite, e, 2, union, court, building, elizabet..."




massau


,node_id,address,name,countries,country_codes,sourceID,valid_until,note,working_address,address_wordlist
810,14049672,"MASSAU, BAHAMAS",NaN,Bahamas,BHS,Panama Papers,The Panama Papers data is current through 2015,NaN,massau bahamas,"[massau, bahamas]"




343nassau


,node_id,address,name,countries,country_codes,sourceID,valid_until,note,working_address,address_wordlist
681,14035227,CB 11.343/Nassau Bahamas,NaN,Bahamas,BHS,Panama Papers,The Panama Papers data is current through 2015,NaN,cb 11 343nassau bahamas,"[cb, 11, 343nassau, bahamas]"


### Shirley

In [48]:
fuzzy_words_df[(fuzzy_words_df['original_value']=='shirley') & (fuzzy_words_df['ratio_score']>60)].sort_values('ratio_score', ascending=False)

,original_index,original_value,match_index,match_value,ratio_score,partial_ratio_score,token_sort_score,token_set_score,jaro_winkler_score
4427,3,shirley,1197,shiriley,93.333333,85.714286,93.333333,93.333333,97.500000
4432,3,shirley,1203,shirly,92.307692,90.909091,92.307692,92.307692,97.142857
4366,3,shirley,1087,shorley,85.714286,85.714286,85.714286,85.714286,92.380952
4424,3,shirley,1193,shitley,85.714286,85.714286,85.714286,85.714286,93.333333
4426,3,shirley,1196,shirely,85.714286,85.714286,85.714286,85.714286,97.142857
4458,3,shirley,1237,shriley,85.714286,85.714286,85.714286,85.714286,96.190476
4429,3,shirley,1199,andshirley,82.352941,100.000000,82.352941,82.352941,90.000000
4380,3,shirley,1119,shirleyand,82.352941,100.000000,82.352941,82.352941,94.000000
4069,3,shirley,525,shirlaw,71.428571,83.333333,71.428571,71.428571,88.571429
4645,3,shirley,1444,haley,66.666667,75.000000,66.666667,66.666667,79.047619


In [49]:
fuzzy_words_df[(fuzzy_words_df['original_value']=='shirley') & (fuzzy_words_df['partial_ratio_score']>60)].sort_values('partial_ratio_score', ascending=False)

,original_index,original_value,match_index,match_value,ratio_score,partial_ratio_score,token_sort_score,token_set_score,jaro_winkler_score
4675,3,shirley,1479,y,25.000000,100.000000,25.000000,25.000000,0.000000
4029,3,shirley,419,e,25.000000,100.000000,25.000000,25.000000,0.000000
4379,3,shirley,1113,l,25.000000,100.000000,25.000000,25.000000,0.000000
4380,3,shirley,1119,shirleyand,82.352941,100.000000,82.352941,82.352941,94.000000
3909,3,shirley,123,s,25.000000,100.000000,25.000000,25.000000,74.285714
4388,3,shirley,1145,i,25.000000,100.000000,25.000000,25.000000,71.428571
4429,3,shirley,1199,andshirley,82.352941,100.000000,82.352941,82.352941,90.000000
4937,3,shirley,1858,hi,44.444444,100.000000,44.444444,44.444444,76.190476
4032,3,shirley,433,r,25.000000,100.000000,25.000000,25.000000,0.000000
4000,3,shirley,355,h,25.000000,100.000000,25.000000,25.000000,71.428571


In [50]:
fuzzy_words_df[(fuzzy_words_df['original_value']=='shirley') & (fuzzy_words_df['token_sort_score']>60)].sort_values('token_sort_score', ascending=False)

,original_index,original_value,match_index,match_value,ratio_score,partial_ratio_score,token_sort_score,token_set_score,jaro_winkler_score
4427,3,shirley,1197,shiriley,93.333333,85.714286,93.333333,93.333333,97.500000
4432,3,shirley,1203,shirly,92.307692,90.909091,92.307692,92.307692,97.142857
4366,3,shirley,1087,shorley,85.714286,85.714286,85.714286,85.714286,92.380952
4424,3,shirley,1193,shitley,85.714286,85.714286,85.714286,85.714286,93.333333
4426,3,shirley,1196,shirely,85.714286,85.714286,85.714286,85.714286,97.142857
4458,3,shirley,1237,shriley,85.714286,85.714286,85.714286,85.714286,96.190476
4429,3,shirley,1199,andshirley,82.352941,100.000000,82.352941,82.352941,90.000000
4380,3,shirley,1119,shirleyand,82.352941,100.000000,82.352941,82.352941,94.000000
4069,3,shirley,525,shirlaw,71.428571,83.333333,71.428571,71.428571,88.571429
4645,3,shirley,1444,haley,66.666667,75.000000,66.666667,66.666667,79.047619


In [51]:
fuzzy_words_df[(fuzzy_words_df['original_value']=='shirley') & (fuzzy_words_df['token_set_score']>60)].sort_values('token_set_score', ascending=False)

,original_index,original_value,match_index,match_value,ratio_score,partial_ratio_score,token_sort_score,token_set_score,jaro_winkler_score
4427,3,shirley,1197,shiriley,93.333333,85.714286,93.333333,93.333333,97.500000
4432,3,shirley,1203,shirly,92.307692,90.909091,92.307692,92.307692,97.142857
4366,3,shirley,1087,shorley,85.714286,85.714286,85.714286,85.714286,92.380952
4424,3,shirley,1193,shitley,85.714286,85.714286,85.714286,85.714286,93.333333
4426,3,shirley,1196,shirely,85.714286,85.714286,85.714286,85.714286,97.142857
4458,3,shirley,1237,shriley,85.714286,85.714286,85.714286,85.714286,96.190476
4429,3,shirley,1199,andshirley,82.352941,100.000000,82.352941,82.352941,90.000000
4380,3,shirley,1119,shirleyand,82.352941,100.000000,82.352941,82.352941,94.000000
4069,3,shirley,525,shirlaw,71.428571,83.333333,71.428571,71.428571,88.571429
4645,3,shirley,1444,haley,66.666667,75.000000,66.666667,66.666667,79.047619


In [52]:
fuzzy_words_df[(fuzzy_words_df['original_value']=='shirley') & (fuzzy_words_df['jaro_winkler_score']>60)].sort_values('jaro_winkler_score', ascending=False)

,original_index,original_value,match_index,match_value,ratio_score,partial_ratio_score,token_sort_score,token_set_score,jaro_winkler_score
4427,3,shirley,1197,shiriley,93.333333,85.714286,93.333333,93.333333,97.500000
4426,3,shirley,1196,shirely,85.714286,85.714286,85.714286,85.714286,97.142857
4432,3,shirley,1203,shirly,92.307692,90.909091,92.307692,92.307692,97.142857
4458,3,shirley,1237,shriley,85.714286,85.714286,85.714286,85.714286,96.190476
4380,3,shirley,1119,shirleyand,82.352941,100.000000,82.352941,82.352941,94.000000
...,...,...,...,...,...,...,...,...,...
4544,3,shirley,1333,sociedad,40.000000,50.000000,40.000000,40.000000,60.119048
4561,3,shirley,1353,property,40.000000,42.857143,40.000000,40.000000,60.119048
4822,3,shirley,1676,highland,40.000000,50.000000,40.000000,40.000000,60.119048
4619,3,shirley,1416,vinicole,40.000000,50.000000,40.000000,40.000000,60.119048


In [53]:
word_list = fuzzy_words_df[(fuzzy_words_df['original_value']=='shirley') & (fuzzy_words_df['jaro_winkler_score']>85)].sort_values('jaro_winkler_score', ascending=False)['match_value']
for word in word_list:
    print(word)
    display(df[df['address_wordlist'].apply(lambda x: word in x)])
    print('\n')

shiriley


,node_id,address,name,countries,country_codes,sourceID,valid_until,note,working_address,address_wordlist
1029,14079987,SUITE E-2; UNION COURT BUILDING; ELIZABETH AVE...,NaN,Bahamas,BHS,Panama Papers,The Panama Papers data is current through 2015,NaN,suite e2 union court building elizabeth avenue...,"[suite, e2, union, court, building, elizabeth,..."




shirely


,node_id,address,name,countries,country_codes,sourceID,valid_until,note,working_address,address_wordlist
1027,14079985,Suite E-2; Union Court Building; Elizabeth Ave...,NaN,Bahamas,BHS,Panama Papers,The Panama Papers data is current through 2015,NaN,suite e 2 union court building elizabeth avenu...,"[suite, e, 2, union, court, building, elizabet..."
1028,14079986,SUITE E-2; UNION COURT BUILDING; ELIZABETH AVE...,NaN,Bahamas,BHS,Panama Papers,The Panama Papers data is current through 2015,NaN,suite e2 union court building elizabeth avenue...,"[suite, e2, union, court, building, elizabeth,..."




shirly


...

,node_id,address,name,countries,country_codes,sourceID,valid_until,note,working_address,address_wordlist
1037,14079997,Suite E - 2; Union Court Building; Elizabeth A...,NaN,Bahamas,BHS,Panama Papers,The Panama Papers data is current through 2015,NaN,suite e 2 union court building elizabeth avenu...,"[suite, e, 2, union, court, building, elizabet..."




shirlaw


,node_id,address,name,countries,country_codes,sourceID,valid_until,note,working_address,address_wordlist
281,24000282,"SHIRLEY STREET, SHIRLAW HOUSE, P.O. BOX N-4839...",NaN,Bahamas,BHS,Bahamas Leaks,The Bahamas Leaks data is current through earl...,NaN,shirley street shirlaw house po box n4839 nass...,"[shirley, street, shirlaw, house, po, box, n48..."
1591,81031328,Higgs & Johnson Corporate Services Ltd.; Shirl...,Higgs & Johnson Corporate Services Ltd.,Bahamas,BHS,Paradise Papers - Appleby,Appleby data is current through 2014,NaN,higgs and johnson corporate services ltd shirl...,"[higgs, and, johnson, corporate, services, ltd..."
1880,33000070,"SHIRLEY STREET, SHIRLAW HOUSE, PO BOX N-4839, ...",NaN,Bahamas,BHS,Paradise Papers - Bahamas corporate registry,Bahamas corporate registry data is current thr...,NaN,shirley street shirlaw house po box n4839 nass...,"[shirley, street, shirlaw, house, po, box, n48..."




shorline


,node_id,address,name,countries,country_codes,sourceID,valid_until,note,working_address,address_wordlist
573,14020532,64; Shorline; Double Road; Freeport; Grand Bah...,NaN,Bahamas,BHS,Panama Papers,The Panama Papers data is current through 2015,NaN,64 shorline double road freeport grand bahamas...,"[64, shorline, double, road, freeport, grand, ..."


# Resources

## Fuzzy Matching

- [Fuzzing matching in pandas with fuzzywuzzy](https://jonathansoma.com/lede/algorithms-2017/classes/fuzziness-matplotlib/fuzzing-matching-in-pandas-with-fuzzywuzzy/)
- [Best Libraries for Fuzzy Matching In Python](https://medium.com/codex/best-libraries-for-fuzzy-matching-in-python-cbb3e0ef87dd)
- [Fuzzy String Matching](https://towardsdatascience.com/fuzzy-string-matching-in-python-68f240d910fe)
- [Fuzzy String Comparison](https://stackoverflow.com/a/28467760)
- [How to do Fuzzy Matching on Pandas Dataframe Column Using Python?](https://www.geeksforgeeks.org/how-to-do-fuzzy-matching-on-pandas-dataframe-column-using-python/)

## Timeit

- [Timeit in Jupyter Notebook](https://linuxhint.com/timeit-jupyter-notebook/)